# EDA

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
data = pd.read_csv('/content/uber.csv')
data.head()

,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count
0,7.5,2015-05-07 19:52:06 UTC,-73.999817,40.738354,-73.999512,40.723217,1
1,7.7,2009-07-17 20:04:56 UTC,-73.994355,40.728225,-73.994710,40.750325,1
2,12.9,2009-08-24 21:45:00 UTC,-74.005043,40.740770,-73.962565,40.772647,1
3,5.3,2009-06-26 08:22:21 UTC,-73.976124,40.790844,-73.965316,40.803349,3
4,16.0,2014-08-28 17:47:00 UTC,-73.925023,40.744085,-73.973082,40.761247,5


In [ ]:
print(data.info())
print(data.describe())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 196024 entries, 0 to 196023
Data columns (total 7 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   fare_amount        196024 non-null  float64
 1   pickup_datetime    196024 non-null  object 
 2   pickup_longitude   196024 non-null  float64
 3   pickup_latitude    196024 non-null  float64
 4   dropoff_longitude  196023 non-null  float64
 5   dropoff_latitude   196023 non-null  float64
 6   passenger_count    196024 non-null  int64  
dtypes: float64(5), int64(1), object(1)
memory usage: 10.5+ MB
None
         fare_amount  pickup_longitude  pickup_latitude  dropoff_longitude  \
count  196024.000000     196024.000000    196024.000000      196023.000000   
mean       11.345803        -73.932369        40.709309         -73.917978   
std         9.796893          5.178129         5.388933           8.506206   
min         0.010000      -1340.648410       -74.015515       -3356.6

In [ ]:
# Limiting the coordinates to New York City
data = data[
    (data['pickup_longitude'] > -74.05) & (data['pickup_longitude'] < -73.7) &
    (data['pickup_latitude'] > 40.58) & (data['pickup_latitude'] < 40.90) &
    (data['dropoff_longitude'] > -74.05) & (data['dropoff_longitude'] < -73.7) &
    (data['dropoff_latitude'] > 40.58) & (data['dropoff_latitude'] < 40.90)
]
data

,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count
0,7.5,2015-05-07 19:52:06 UTC,-73.999817,40.738354,-73.999512,40.723217,1
1,7.7,2009-07-17 20:04:56 UTC,-73.994355,40.728225,-73.994710,40.750325,1
2,12.9,2009-08-24 21:45:00 UTC,-74.005043,40.740770,-73.962565,40.772647,1
3,5.3,2009-06-26 08:22:21 UTC,-73.976124,40.790844,-73.965316,40.803349,3
4,16.0,2014-08-28 17:47:00 UTC,-73.925023,40.744085,-73.973082,40.761247,5
...,...,...,...,...,...,...,...
196019,3.0,2012-10-28 10:49:00 UTC,-73.987042,40.739367,-73.986525,40.740297,1
196020,7.5,2014-03-14 01:09:00 UTC,-73.984722,40.736837,-74.006672,40.739620,1
196021,30.9,2009-06-29 00:42:00 UTC,-73.986017,40.756487,-73.858957,40.692588,2
196022,14.5,2015-05-20 14:56:25 UTC,-73.997124,40.725452,-73.983215,40.695416,1


In [ ]:
data.describe()

,fare_amount,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count
count,194987.000000,194987.000000,194987.000000,194987.000000,194987.000000,194987.000000
mean,11.171091,-73.975460,40.750815,-73.974212,40.751175,1.687333
std,9.109697,0.034239,0.026804,0.032891,0.030281,1.304080
min,0.010000,-74.049495,40.581745,-74.049012,40.580783,1.000000
25%,6.000000,-73.992255,40.736492,-73.991536,40.735506,1.000000
50%,8.500000,-73.982110,40.753315,-73.980513,40.753788,1.000000
75%,12.500000,-73.968392,40.767529,-73.965428,40.768317,2.000000
max,499.000000,-73.702735,40.897572,-73.700650,40.899914,6.000000


In [ ]:
data.isna().sum()

fare_amount          0
pickup_datetime      0
pickup_longitude     0
pickup_latitude      0
dropoff_longitude    0
dropoff_latitude     0
passenger_count      0
dtype: int64

**Calculate Distance:**

The most common approach is to compute the haversine distance between the pickup and dropoff points. This gives the "as-the-crow-flies" distance, which, while not perfect (since taxis don't fly and have to follow roads), can be a good approximation.

In [ ]:
def haversine_distance(row):
    R = 6371.0  # Earth's radius in kilometers

    lat1 = np.radians(row['pickup_latitude'])
    lon1 = np.radians(row['pickup_longitude'])
    lat2 = np.radians(row['dropoff_latitude'])
    lon2 = np.radians(row['dropoff_longitude'])

    dlat = lat2 - lat1
    dlon = lon2 - lon1

    a = np.sin(dlat / 2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon / 2)**2
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a))

    distance = R * c
    return distance

# Apply the function to each row in the DataFrame
data['distance'] = data.apply(haversine_distance, axis=1)
data.describe()

,fare_amount,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count,distance
count,194987.000000,194987.000000,194987.000000,194987.000000,194987.000000,194987.000000,194987.000000
mean,11.171091,-73.975460,40.750815,-73.974212,40.751175,1.687333,3.276204
std,9.109697,0.034239,0.026804,0.032891,0.030281,1.304080,3.486564
min,0.010000,-74.049495,40.581745,-74.049012,40.580783,1.000000,0.000000
25%,6.000000,-73.992255,40.736492,-73.991536,40.735506,1.000000,1.255912
50%,8.500000,-73.982110,40.753315,-73.980513,40.753788,1.000000,2.152393
75%,12.500000,-73.968392,40.767529,-73.965428,40.768317,2.000000,3.886467
max,499.000000,-73.702735,40.897572,-73.700650,40.899914,6.000000,29.673392


In [ ]:
# Removing rows with distance = 0
data = data[data['distance']!=0]
data.describe()

,fare_amount,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count,distance
count,193047.000000,193047.000000,193047.000000,193047.000000,193047.000000,193047.000000,193047.000000
mean,11.179215,-73.975609,40.750866,-73.974348,40.751230,1.687014,3.309128
std,9.073149,0.034004,0.026646,0.032638,0.030173,1.303451,3.488459
min,0.010000,-74.049495,40.581745,-74.049012,40.580783,1.000000,0.000084
25%,6.000000,-73.992270,40.736520,-73.991547,40.735540,1.000000,1.279270
50%,8.500000,-73.982135,40.753325,-73.980548,40.753803,1.000000,2.174996
75%,12.500000,-73.968512,40.767546,-73.965598,40.768331,2.000000,3.913756
max,499.000000,-73.702735,40.897572,-73.700650,40.899914,6.000000,29.673392


In [ ]:
data.loc[:, 'pickup_datetime'] = pd.to_datetime(data['pickup_datetime'])
data.loc[:, 'hour'] = data['pickup_datetime'].dt.hour
data.loc[:, 'day'] = data['pickup_datetime'].dt.day
data.loc[:, 'month'] = data['pickup_datetime'].dt.month
data.loc[:, 'year'] = data['pickup_datetime'].dt.year
data.loc[:, 'weekday'] = data['pickup_datetime'].dt.weekday
data

<ipython-input-9-8df0fe3d8310>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.loc[:, 'pickup_datetime'] = pd.to_datetime(data['pickup_datetime'])
<ipython-input-9-8df0fe3d8310>:1: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  data.loc[:, 'pickup_datetime'] = pd.to_datetime(data['pickup_datetime'])
<ipython-input-9-8df0fe3d8310>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: ht

,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count,distance,hour,day,month,year,weekday
0,7.5,2015-05-07 19:52:06+00:00,-73.999817,40.738354,-73.999512,40.723217,1,1.683323,19,7,5,2015,3
1,7.7,2009-07-17 20:04:56+00:00,-73.994355,40.728225,-73.994710,40.750325,1,2.457590,20,17,7,2009,4
2,12.9,2009-08-24 21:45:00+00:00,-74.005043,40.740770,-73.962565,40.772647,1,5.036377,21,24,8,2009,0
3,5.3,2009-06-26 08:22:21+00:00,-73.976124,40.790844,-73.965316,40.803349,3,1.661683,8,26,6,2009,4
4,16.0,2014-08-28 17:47:00+00:00,-73.925023,40.744085,-73.973082,40.761247,5,4.475450,17,28,8,2014,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...
196019,3.0,2012-10-28 10:49:00+00:00,-73.987042,40.739367,-73.986525,40.740297,1,0.112210,10,28,10,2012,6
196020,7.5,2014-03-14 01:09:00+00:00,-73.984722,40.736837,-74.006672,40.739620,1,1.875050,1,14,3,2014,4
196021,30.9,2009-06-29 00:42:00+00:00,-73.986017,40.756487,-73.858957,40.692588,2,12.850319,0,29,6,2009,0
196022,14.5,2015-05-20 14:56:25+00:00,-73.997124,40.725452,-73.983215,40.695416,1,3.539715,14,20,5,2015,2


In [ ]:
data['weekend'] = data['weekday'].apply(lambda x: 1 if x >=5 else 0)
data

<ipython-input-10-0b0dc7fe8ca8>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['weekend'] = data['weekday'].apply(lambda x: 1 if x >=5 else 0)


,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count,distance,hour,day,month,year,weekday,weekend
0,7.5,2015-05-07 19:52:06+00:00,-73.999817,40.738354,-73.999512,40.723217,1,1.683323,19,7,5,2015,3,0
1,7.7,2009-07-17 20:04:56+00:00,-73.994355,40.728225,-73.994710,40.750325,1,2.457590,20,17,7,2009,4,0
2,12.9,2009-08-24 21:45:00+00:00,-74.005043,40.740770,-73.962565,40.772647,1,5.036377,21,24,8,2009,0,0
3,5.3,2009-06-26 08:22:21+00:00,-73.976124,40.790844,-73.965316,40.803349,3,1.661683,8,26,6,2009,4,0
4,16.0,2014-08-28 17:47:00+00:00,-73.925023,40.744085,-73.973082,40.761247,5,4.475450,17,28,8,2014,3,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
196019,3.0,2012-10-28 10:49:00+00:00,-73.987042,40.739367,-73.986525,40.740297,1,0.112210,10,28,10,2012,6,1
196020,7.5,2014-03-14 01:09:00+00:00,-73.984722,40.736837,-74.006672,40.739620,1,1.875050,1,14,3,2014,4,0
196021,30.9,2009-06-29 00:42:00+00:00,-73.986017,40.756487,-73.858957,40.692588,2,12.850319,0,29,6,2009,0,0
196022,14.5,2015-05-20 14:56:25+00:00,-73.997124,40.725452,-73.983215,40.695416,1,3.539715,14,20,5,2015,2,0


In [ ]:
def time_of_day(hour):
    if 5 <= hour < 12:
        return 'morning'
    elif 12 <= hour < 17:
        return 'afternoon'
    elif 17 <= hour < 21:
        return 'evening'
    else:
        return 'night'

data['time_of_day'] = data['hour'].apply(time_of_day)
data

<ipython-input-11-7d2d5c0ae56b>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['time_of_day'] = data['hour'].apply(time_of_day)


,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count,distance,hour,day,month,year,weekday,weekend,time_of_day
0,7.5,2015-05-07 19:52:06+00:00,-73.999817,40.738354,-73.999512,40.723217,1,1.683323,19,7,5,2015,3,0,evening
1,7.7,2009-07-17 20:04:56+00:00,-73.994355,40.728225,-73.994710,40.750325,1,2.457590,20,17,7,2009,4,0,evening
2,12.9,2009-08-24 21:45:00+00:00,-74.005043,40.740770,-73.962565,40.772647,1,5.036377,21,24,8,2009,0,0,night
3,5.3,2009-06-26 08:22:21+00:00,-73.976124,40.790844,-73.965316,40.803349,3,1.661683,8,26,6,2009,4,0,morning
4,16.0,2014-08-28 17:47:00+00:00,-73.925023,40.744085,-73.973082,40.761247,5,4.475450,17,28,8,2014,3,0,evening
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
196019,3.0,2012-10-28 10:49:00+00:00,-73.987042,40.739367,-73.986525,40.740297,1,0.112210,10,28,10,2012,6,1,morning
196020,7.5,2014-03-14 01:09:00+00:00,-73.984722,40.736837,-74.006672,40.739620,1,1.875050,1,14,3,2014,4,0,night
196021,30.9,2009-06-29 00:42:00+00:00,-73.986017,40.756487,-73.858957,40.692588,2,12.850319,0,29,6,2009,0,0,night
196022,14.5,2015-05-20 14:56:25+00:00,-73.997124,40.725452,-73.983215,40.695416,1,3.539715,14,20,5,2015,2,0,afternoon


In [ ]:
data_columns_removed = data.drop(columns=['pickup_datetime', 'pickup_longitude', 'pickup_latitude', 'dropoff_longitude', 'dropoff_latitude'])
data_columns_removed

,fare_amount,passenger_count,distance,hour,day,month,year,weekday,weekend,time_of_day
0,7.5,1,1.683323,19,7,5,2015,3,0,evening
1,7.7,1,2.457590,20,17,7,2009,4,0,evening
2,12.9,1,5.036377,21,24,8,2009,0,0,night
3,5.3,3,1.661683,8,26,6,2009,4,0,morning
4,16.0,5,4.475450,17,28,8,2014,3,0,evening
...,...,...,...,...,...,...,...,...,...,...
196019,3.0,1,0.112210,10,28,10,2012,6,1,morning
196020,7.5,1,1.875050,1,14,3,2014,4,0,night
196021,30.9,2,12.850319,0,29,6,2009,0,0,night
196022,14.5,1,3.539715,14,20,5,2015,2,0,afternoon


In [ ]:
time_of_day_dummies = pd.get_dummies(data_columns_removed['time_of_day'])
data_columns_removed = pd.concat([data_columns_removed, time_of_day_dummies], axis=1)
data_columns_removed.drop('time_of_day', axis=1,  inplace=True)
data_columns_removed

,fare_amount,passenger_count,distance,hour,day,month,year,weekday,weekend,afternoon,evening,morning,night
0,7.5,1,1.683323,19,7,5,2015,3,0,0,1,0,0
1,7.7,1,2.457590,20,17,7,2009,4,0,0,1,0,0
2,12.9,1,5.036377,21,24,8,2009,0,0,0,0,0,1
3,5.3,3,1.661683,8,26,6,2009,4,0,0,0,1,0
4,16.0,5,4.475450,17,28,8,2014,3,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
196019,3.0,1,0.112210,10,28,10,2012,6,1,0,0,1,0
196020,7.5,1,1.875050,1,14,3,2014,4,0,0,0,0,1
196021,30.9,2,12.850319,0,29,6,2009,0,0,0,0,0,1
196022,14.5,1,3.539715,14,20,5,2015,2,0,1,0,0,0


In [ ]:
from sklearn.preprocessing import StandardScaler

stdscale = StandardScaler()
stdscale.fit(data_columns_removed[['fare_amount', 'distance']])
data_columns_removed[['fare_amount', 'distance']] = stdscale.transform(data_columns_removed[['fare_amount', 'distance']])
data_columns_removed


,fare_amount,passenger_count,distance,hour,day,month,year,weekday,weekend,afternoon,evening,morning,night
0,-0.405507,1,-0.466054,19,7,5,2015,3,0,0,1,0,0
1,-0.383464,1,-0.244102,20,17,7,2009,4,0,0,1,0,0
2,0.189657,1,0.495134,21,24,8,2009,0,0,0,0,0,1
3,-0.647981,3,-0.472257,8,26,6,2009,4,0,0,0,1,0
4,0.531326,5,0.334338,17,28,8,2014,3,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
196019,-0.901477,1,-0.916429,10,28,10,2012,6,1,0,0,1,0
196020,-0.405507,1,-0.411093,1,14,3,2014,4,0,0,0,0,1
196021,2.173538,2,2.735081,0,29,6,2009,0,0,0,0,0,1
196022,0.366002,1,0.066100,14,20,5,2015,2,0,1,0,0,0


In [ ]:
from sklearn.model_selection import train_test_split

X = data_columns_removed.drop('fare_amount', axis=1)
y = data_columns_removed['fare_amount']

# Splitting the data into training, validation, and testing sets (80%, 10%, 10%)
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.2, random_state=42)
X_valid, X_test, y_valid, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

print("Training Data Shape:", X_train.shape)
print("Validation Data Shape:", X_valid.shape)
print("Testing Data Shape:", X_test.shape)


Training Data Shape: (154437, 12)
Validation Data Shape: (19305, 12)
Testing Data Shape: (19305, 12)


# Deep Neural Networks

In [ ]:
!pip install tensorflow_addons

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 612.1/612.1 kB 6.0 MB/s eta 0:00:00


In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow_addons.optimizers import AdamW, Lookahead

model = keras.models.Sequential([
    tf.keras.layers.Dense(100, activation='swish', kernel_initializer='he_normal', input_shape=(12,)),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.BatchNormalization(),

    tf.keras.layers.Dense(75, activation='swish', kernel_initializer='he_normal'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.BatchNormalization(),

    tf.keras.layers.Dense(50, activation='swish', kernel_initializer='he_normal'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.BatchNormalization(),

    tf.keras.layers.Dense(40, activation='swish', kernel_initializer='he_normal'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.BatchNormalization(),

    tf.keras.layers.Dense(25, activation='swish', kernel_initializer='he_normal'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.BatchNormalization(),

    tf.keras.layers.Dense(10, activation='swish', kernel_initializer='he_normal'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.BatchNormalization(),

    tf.keras.layers.Dense(1)
])
model.summary()


/usr/local/lib/python3.10/dist-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 100)               1300      
                                                                 
 dropout (Dropout)           (None, 100)               0         
                                                                 
 batch_normalization (BatchN  (None, 100)              400       
 ormalization)                                                   
                                                                 
 dense_1 (Dense)             (None, 75)                7575      
                                                                 
 dropout_1 (Dropout)         (None, 75)                0         
                                                                 
 batch_normalization_1 (Batc  (None, 75)               300       
 hNormalization)                                        

In [ ]:
# Weight decay and optimizer
weight_decay = 1e-4
optimizer = AdamW(learning_rate=1e-3, weight_decay=weight_decay)

# Compile the model
model.compile(optimizer=optimizer, loss='mse', metrics=['mae'])

# Early stopping
early_stopping = EarlyStopping(patience=10, restore_best_weights=True)

model.fit(X_train, y_train, epochs=50, validation_data=(X_valid, y_valid), callbacks=[early_stopping])

Epoch 1/50
4827/4827 [==============================] - 44s 8ms/step - loss: 1.0470 - mae: 0.6564 - val_loss: 0.9723 - val_mae: 0.6376
Epoch 2/50
4827/4827 [==============================] - 24s 5ms/step - loss: 0.6509 - mae: 0.4835 - val_loss: 0.4357 - val_mae: 0.3859
Epoch 3/50
4827/4827 [==============================] - 26s 5ms/step - loss: 0.5222 - mae: 0.4236 - val_loss: 6.8299 - val_mae: 2.5544
Epoch 4/50
4827/4827 [==============================] - 35s 7ms/step - loss: 0.5142 - mae: 0.4211 - val_loss: 5.3163 - val_mae: 2.2564
Epoch 5/50
4827/4827 [==============================] - 25s 5ms/step - loss: 0.5115 - mae: 0.4196 - val_loss: 10.5372 - val_mae: 3.1684
Epoch 6/50
4827/4827 [==============================] - 28s 6ms/step - loss: 0.5070 - mae: 0.4194 - val_loss: 19.7402 - val_mae: 4.3663
Epoch 7/50
4827/4827 [==============================] - 26s 5ms/step - loss: 0.5087 - mae: 0.4184 - val_loss: 2.2013 - val_mae: 1.4222
Epoch 8/50
4827/4827 [==============================]

In [ ]:
model.evaluate(X_test, y_test)

604/604 [==============================] - 2s 3ms/step - loss: 0.4350 - mae: 0.3860


[0.4350283741950989, 0.38604557514190674]

In [ ]:
!pip install keras-tuner


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 176.1/176.1 kB 2.7 MB/s eta 0:00:00


In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow_addons.optimizers import AdamW, Lookahead
from kerastuner import HyperModel, RandomSearch
from tensorflow.keras.callbacks import EarlyStopping

class UberFareHyperModel(HyperModel):
    def __init__(self, input_shape):
        self.input_shape = input_shape

    def build(self, hp):
        model = keras.models.Sequential()

        # Input layer
        model.add(Dense(units=hp.Int('units_input', min_value=50, max_value=150, step=10),
                        activation='swish',
                        kernel_initializer='he_normal',
                        input_shape=self.input_shape))
        model.add(Dropout(rate=hp.Float('dropout_input', 0.0, 0.5, step=0.1)))
        model.add(BatchNormalization())

        # Hidden layers
        for i in range(hp.Int('num_layers', 1, 5)):
            model.add(Dense(units=hp.Int(f'units_{i}', min_value=25, max_value=100, step=5),
                            activation='swish',
                            kernel_initializer='he_normal'))
            model.add(Dropout(rate=hp.Float(f'dropout_{i}', 0.0, 0.5, step=0.1)))
            model.add(BatchNormalization())

        # Output layer
        model.add(Dense(1))

        # Compile model
        optimizer_choice = hp.Choice('optimizer', ['adam', 'adamw'])
        if optimizer_choice == 'adam':
            optimizer = keras.optimizers.Adam(
            learning_rate=hp.Float('learning_rate', 1e-4, 1e-2, sampling='log'))
        else:
            optimizer = AdamW(
            learning_rate=hp.Float('learning_rate', 1e-4, 1e-2, sampling='log'),
            weight_decay=hp.Float('weight_decay', 1e-5, 1e-2, sampling='log'))  # Added weight_decay hyperparameter

        model.compile(optimizer=optimizer, loss='mse', metrics=['mae'])

        return model

input_shape = (12,)
hypermodel = UberFareHyperModel(input_shape)

tuner = RandomSearch(
    hypermodel,
    objective='val_mae',
    max_trials=30,
    executions_per_trial=2,
    directory='uber_fare_tuning',
    project_name='keras_tuner_demo'
)

tuner.search_space_summary()

# Assuming you have your training data in X_train, y_train and validation data in X_val, y_val
tuner.search(X_train, y_train, epochs=50, validation_data=(X_valid, y_valid), callbacks=[EarlyStopping(patience=5)])

# Get the optimal hyperparameters
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]


Trial 30 Complete [00h 05m 33s]
val_mae: 0.34812071174383163

Best val_mae So Far: 0.27774037420749664
Total elapsed time: 04h 28m 28s


NameError: ignored

In [ ]:
model = tuner.hypermodel.build(best_hps)
history = model.fit(X_train, y_train, epochs=50, validation_data=(X_valid, y_valid), callbacks=[EarlyStopping(patience=5)])


Epoch 1/50
4827/4827 [==============================] - 44s 8ms/step - loss: 0.9869 - mae: 0.6249 - val_loss: 0.5149 - val_mae: 0.4650
Epoch 2/50
4827/4827 [==============================] - 33s 7ms/step - loss: 0.5140 - mae: 0.4010 - val_loss: 0.3847 - val_mae: 0.3651
Epoch 3/50
4827/4827 [==============================] - 35s 7ms/step - loss: 0.4425 - mae: 0.3683 - val_loss: 0.3792 - val_mae: 0.4865
Epoch 4/50
4827/4827 [==============================] - 35s 7ms/step - loss: 0.4332 - mae: 0.3621 - val_loss: 0.5860 - val_mae: 0.6710
Epoch 5/50
4827/4827 [==============================] - 35s 7ms/step - loss: 0.4251 - mae: 0.3596 - val_loss: 1.1216 - val_mae: 0.8287
Epoch 6/50
4827/4827 [==============================] - 34s 7ms/step - loss: 0.4204 - mae: 0.3579 - val_loss: 0.9270 - val_mae: 0.8931
Epoch 7/50
4827/4827 [==============================] - 33s 7ms/step - loss: 0.4205 - mae: 0.3561 - val_loss: 0.3360 - val_mae: 0.3271
Epoch 8/50
4827/4827 [==============================] -

In [ ]:
model.evaluate(X_test, y_test)

604/604 [==============================] - 2s 3ms/step - loss: 0.4927 - mae: 0.6070


[0.4926953911781311, 0.6070067882537842]

In [ ]:
best_hps.values

{'units_input': 110,
 'dropout_input': 0.1,
 'num_layers': 5,
 'units_0': 65,
 'dropout_0': 0.2,
 'optimizer': 'adam',
 'learning_rate': 0.0006721540921761134,
 'units_1': 80,
 'dropout_1': 0.4,
 'units_2': 80,
 'dropout_2': 0.2,
 'units_3': 70,
 'dropout_3': 0.2,
 'units_4': 100,
 'dropout_4': 0.30000000000000004,
 'weight_decay': 3.6420971529420396e-05}

Input Layer:
- Units: 110
- Dropout: 10%

Hidden Layers:

- 1st Hidden Layer:
 - Units: 65
 - Dropout: 20%
- 2nd Hidden Layer:
 - Units: 80
 - Dropout: 40%
- 3rd Hidden Layer:
 - Units: 80
 - Dropout: 20%
- 4th Hidden Layer:
 - Units: 70
 - Dropout: 20%
- 5th Hidden Layer:
 - Units: 100
 - Dropout: 30%

Optimizer:
- Type: Adam
- Learning Rate: ~0.000672
- Weight Decay: ~3.64e-05

In [ ]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_11 (Dense)            (None, 110)               1430      
                                                                 
 dropout_9 (Dropout)         (None, 110)               0         
                                                                 
 batch_normalization_9 (Batc  (None, 110)              440       
 hNormalization)                                                 
                                                                 
 dense_12 (Dense)            (None, 65)                7215      
                                                                 
 dropout_10 (Dropout)        (None, 65)                0         
                                                                 
 batch_normalization_10 (Bat  (None, 65)               260       
 chNormalization)                                     